In [18]:
import pandas as pd

url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_1ff034cd9d5243feb7ffb3923815a393.csv"
df=pd.read_csv(url)
df.head()

,COU,Country,POL,Pollutant,VAR,Variable,YEA,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1990,1990,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,424998.381,NaN,NaN
1,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1991,1991,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,426015.210,NaN,NaN
2,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1992,1992,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430216.380,NaN,NaN
3,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1993,1993,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430612.556,NaN,NaN
4,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1994,1994,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430653.332,NaN,NaN


###### Let's get to know the dataset first.

In [19]:
import numpy as np

np.unique(df["Country"])

array(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada',
       'Chile', "China (People's Republic of)", 'Colombia', 'Costa Rica',
       'Czech Republic', 'Denmark', 'Estonia',
       'European Union (28 countries)', 'Finland', 'France', 'Germany',
       'Greece', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland',
       'Israel', 'Italy', 'Japan', 'Korea', 'Latvia', 'Lithuania',
       'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand', 'Norway',
       'OECD - Europe', 'OECD - Total', 'Poland', 'Portugal', 'Russia',
       'Slovak Republic', 'Slovenia', 'South Africa', 'Spain', 'Sweden',
       'Switzerland', 'Turkey', 'United Kingdom', 'United States'],
      dtype=object)

In [20]:
# Drop the rows for "European Union", "OECD - Europe" and "OECD - Total" to avoid double-counting. 
# If we include these organizations and the individual memeber countries, that will be double-counting.
df=df.drop(df[df["Country"]=='European Union (28 countries)'].index)
df=df.drop(df[df["Country"]=='OECD - Europe'].index)
df=df.drop(df[df["Country"]=='OECD - Total'].index)
# Verify that those summary rows had been dropped.
np.unique(df["Country"])

array(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada',
       'Chile', "China (People's Republic of)", 'Colombia', 'Costa Rica',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'India', 'Indonesia',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Korea', 'Latvia',
       'Lithuania', 'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand',
       'Norway', 'Poland', 'Portugal', 'Russia', 'Slovak Republic',
       'Slovenia', 'South Africa', 'Spain', 'Sweden', 'Switzerland',
       'Turkey', 'United Kingdom', 'United States'], dtype=object)

In [21]:
# Let's see what is in the "Pollutant" column.
np.unique(df["Pollutant"])

array(['Carbon dioxide', 'Greenhouse gases', 'Hydrofluorocarbons',
       'Methane', 'Nitrogen trifluoride', 'Nitrous oxide',
       'Perfluorocarbons', 'Sulphur hexafluoride',
       'Unspecified mix of HFCs and PFCs'], dtype=object)

###### The "Pollutant" column includes rows for "Greenhouse gases" (the overall name) as well as rows for each gas. I want to analyze emission of all the greenhouse gases by countries. Hence, I want the rows for "Greenhouse gases" (the overall name), not the specific gas.

In [22]:
df=df[df["Pollutant"]=='Greenhouse gases']
np.unique(df["Pollutant"])

array(['Greenhouse gases'], dtype=object)

In [23]:
# Let's see what is in the "VAR" column.
np.unique(df["VAR"])

array(['AFOLU', 'AGR', 'AGR_P', 'ENER', 'ENER_CO2', 'ENER_CO2_P',
       'ENER_FU', 'ENER_FU_P', 'ENER_IND', 'ENER_IND_P', 'ENER_MANUF',
       'ENER_MANUF_P', 'ENER_OSECT', 'ENER_OSECT_P', 'ENER_OTH',
       'ENER_OTH_P', 'ENER_P', 'ENER_TRANS', 'ENER_TRANS_P', 'GHG_CAP',
       'GHG_GDP', 'INDEX_1990', 'INDEX_2000', 'IND_PROC', 'IND_PROC_P',
       'LULUCF', 'OTH', 'OTH_P', 'TOTAL', 'TOTAL_LULU', 'WAS', 'WAS_P'],
      dtype=object)

##### Since I want to analyze total greenhouse gases emission, I want the "TOTAL" rows only.

In [24]:
df=df[df["VAR"]=="TOTAL"]
np.unique(df["VAR"])

array(['TOTAL'], dtype=object)

In [25]:
# Let's see what is in the "Variable" column.
np.unique(df["Variable"])

array(['Total  emissions excluding LULUCF'], dtype=object)

###### I have dropped all the rows that I don't need.

In [26]:
# Let's output the column names.
df.columns

Index(['COU', 'Country', 'POL', 'Pollutant', 'VAR', 'Variable', 'YEA', 'Year',
       'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')

##### For the analysis I want to perform, I only need the "COU" column, which consists of the three-alphabet country codes, the "Country" column, which has the country names, the "Value" column, which consists of the total greenhouse gases emission for the countries and the "Year" column.

In [27]:
df=df[["COU","Country","Value","Year"]]
df.columns

Index(['COU', 'Country', 'Value', 'Year'], dtype='object')

###### An animated map to visualize the analytical results as a time series.

In [28]:
import plotly.express as px

fig = px.choropleth(df,
                    locations ="COU",
                    color ="Value",
                    hover_name ="Country", 
                    color_continuous_scale = px.colors.sequential.Plasma,
                    scope ="world",
                    labels={'Value':'Greenhouse gases in millions of metric tons'},
                    animation_frame ="Year")
fig.update_layout(title="Greenhouse gases emission by country, 1990-2018")
fig.show()

In [12]:
# Define a function to rank the countries based on the percentage of greenhouse gases emission.
def ghg_rank(n):
  yearn=df[df["Year"]==n]
  "Group the data by year, calcualte each country's contribution to global ghg each year, then rank them based on the % of global ghg that they contributed."
  year_sums = yearn.groupby(['Year',"Country"]).agg({'Value': 'sum'})
  year_sums = year_sums.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum())).rename(columns={"Value":"Greenhouse gases emission(%)"}).reset_index()
  comparison=year_sums.sort_values(by=["Greenhouse gases emission(%)"],ascending=False)
  fig = px.bar(comparison, x="Country", y="Greenhouse gases emission(%)",
             color='Country',
             height=400)
  fig.update_layout(title=f"Greenhouse gases emission by country, {n}")
  return fig.show()

###### Static bar charts to visualize the analytical results year by year.

In [13]:
# Use a loop for go through all the years for which data were available. 
for i in np.arange(1990,2019):
  ghg_rank(i)

In [36]:
year_sums[year_sums["Country"]=="United States"]

,Year,Country,Greenhouse gases emission(%)
39,1990,United States,32.833248
78,1991,United States,33.398641
117,1992,United States,34.767265
156,1993,United States,35.484585
198,1994,United States,27.618616
237,1995,United States,36.118630
277,1996,United States,36.304935
316,1997,United States,36.751033
355,1998,United States,37.103819
394,1999,United States,37.207442
